In [3]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer as SIA
from pathlib import Path
from datetime import datetime, timedelta

%matplotlib inline


Bad key text.latex.preview in file c:\Users\gunin\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file c:\Users\gunin\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file c:\Users\gunin\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpeg is saved, 

In [4]:
file = Path('bitcoin_data.csv')
btc_df = pd.read_csv(file)
btc_df

,date,open,close,low
0,2022-05-27 10:59:59.999,41102.29,40939.11,40918.78
1,2022-05-27 11:59:59.999,40956.02,40585.27,40497.27
2,2022-05-27 12:59:59.999,40583.16,40646.82,40433.17
3,2022-05-27 13:59:59.999,40643.60,40686.85,40618.18
4,2022-05-27 14:59:59.999,40688.24,40483.86,40463.15
...,...,...,...,...
116,2022-06-01 06:59:59.999,44074.70,44260.02,43948.17
117,2022-06-01 07:59:59.999,44268.13,44172.57,44076.02
118,2022-06-01 08:59:59.999,44112.24,44381.28,44079.14
119,2022-06-01 09:59:59.999,44380.00,44260.58,44230.74


In [5]:
new_date = [datetime.strptime(i, "%Y-%m-%d %H:%M:%S.%f") for i in btc_df["date"]]
btc_df["datetime"] = new_date
btc_df = btc_df.drop("date", axis=1)

now = datetime.now()

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)+timedelta(hours=t.minute//30))

btc_df['datetime'] = btc_df['datetime'].apply(lambda t: hour_rounder(t))

btc_df.reset_index()
btc_df = btc_df.set_index("datetime")

In [6]:
file3 = Path('../datasets/btc_dataset_hourly.csv')
btc_tw_df = pd.read_csv(file3)
btc_tw_df.reset_index()


,index,time,nltk_polarity
0,0,2022-05-27 02:00:00+00:00,0.411728
1,1,2022-05-27 03:00:00+00:00,NaN
2,2,2022-05-27 04:00:00+00:00,0.185771
3,3,2022-05-27 05:00:00+00:00,0.432900
4,4,2022-05-27 06:00:00+00:00,NaN
...,...,...,...
137,137,2022-06-01 19:00:00+00:00,0.272658
138,138,2022-06-01 20:00:00+00:00,NaN
139,139,2022-06-01 21:00:00+00:00,0.156100
140,140,2022-06-01 22:00:00+00:00,0.312003


In [7]:
# Create new DF
btc_df = btc_df.drop(columns=['open', 'low'])

# Calculate the hourly returns using the closing prices and the pct_change function
btc_df["actual_returns"] = btc_df["close"].pct_change()
# Define a window size of 50 & 100
short_window = 50
long_window = 100

# Create a simple moving average (SMA) using the short_window and assign this to a new columns called sma_fast
#btc_df["sma_50"] = btc_df["close"].rolling(window=short_window).mean()
#btc_df["sma_100"] = btc_df["close"].rolling(window=long_window).mean()
btc_df

,close,actual_returns
datetime,,
2022-05-27 11:00:00,40939.11,NaN
2022-05-27 12:00:00,40585.27,-0.008643
2022-05-27 13:00:00,40646.82,0.001517
2022-05-27 14:00:00,40686.85,0.000985
2022-05-27 15:00:00,40483.86,-0.004989
...,...,...
2022-06-01 07:00:00,44260.02,0.004307
2022-06-01 08:00:00,44172.57,-0.001976
2022-06-01 09:00:00,44381.28,0.004725


In [8]:
new_date_1 = [datetime.strptime(i, "%Y-%m-%d %H:%M:%S%z") for i in btc_tw_df["time"]]
btc_tw_df["datetime"] = new_date_1
btc_tw_df = btc_tw_df.drop("time", axis=1)

now = datetime.now()

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)+timedelta(hours=t.minute//30))

btc_tw_df["datetime"] = btc_tw_df["datetime"].apply(lambda t: hour_rounder(t))

btc_tw_df['datetime'].apply(lambda x: x.replace(tzinfo=None))

btc_tw_df.reset_index()
btc_tw_df = btc_tw_df.set_index("datetime")

btc_tw_df

,nltk_polarity
datetime,
2022-05-27 02:00:00+00:00,0.411728
2022-05-27 03:00:00+00:00,NaN
2022-05-27 04:00:00+00:00,0.185771
2022-05-27 05:00:00+00:00,0.432900
2022-05-27 06:00:00+00:00,NaN
...,...
2022-06-01 19:00:00+00:00,0.272658
2022-06-01 20:00:00+00:00,NaN
2022-06-01 21:00:00+00:00,0.156100


In [9]:
btc_df.index = pd.to_datetime(btc_df.index).tz_localize('Etc/UCT')
combined_df = pd.concat([btc_df, btc_tw_df], axis=1)


In [10]:
combined_df = combined_df.dropna()

In [11]:
combined_df.shape

(82, 3)

In [12]:
# Create a new column in the btc_df called signal setting its value to zero.
combined_df["Signal"] = 0

# Create the signal to buy
combined_df.loc[(btc_df["actual_returns"] >= 0) & (combined_df['nltk_polarity'] > 0), "Signal"] = 1

# Create the signal to sell
combined_df.loc[(btc_df["actual_returns"] < 0) & (combined_df['nltk_polarity'] < 0), "Signal"] = -1


set(combined_df["Signal"])

{0, 1}

In [13]:
combined_df

,close,actual_returns,nltk_polarity,Signal
datetime,,,,
2022-05-27 13:00:00+00:00,40646.82,0.001517,0.221712,1
2022-05-27 14:00:00+00:00,40686.85,0.000985,0.204893,1
2022-05-27 15:00:00+00:00,40483.86,-0.004989,0.336443,0
2022-05-27 17:00:00+00:00,40641.79,0.006826,0.225527,1
2022-05-27 18:00:00+00:00,40621.65,-0.000496,0.261405,0
...,...,...,...,...
2022-06-01 05:00:00+00:00,44097.58,-0.012457,0.375421,0
2022-06-01 06:00:00+00:00,44070.23,-0.000620,0.533262,0
2022-06-01 07:00:00+00:00,44260.02,0.004307,0.359507,1


In [19]:
query = "bitcoin"
tweets = []
limit = 15000


for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content])
        
df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])


In [20]:
df

,Date,User,Tweet
0,2022-06-07 09:16:11+00:00,mrtnmrtnbld,@rovercrc I TOLD YA WE HIT 10K FOR BITCOIN YAS...
1,2022-06-07 09:16:11+00:00,MinerSalesUK,The best entry level #ASICMiner you can buy w...
2,2022-06-07 09:16:10+00:00,Akira27831558,@lukkOneM @matpizzini @markrobots Se proprio t...
3,2022-06-07 09:16:09+00:00,bmurphypointman,#affiliate #tumblr #twitter #facebook #instagr...
4,2022-06-07 09:16:08+00:00,bmurphypointman,#affiliate #bitcoin #tumblr #twitter #facebook...
...,...,...,...
14995,2022-06-07 02:28:26+00:00,exchange_whale,"🐳 1,145,224 #USDT worth of #Bitcoin were sold ..."
14996,2022-06-07 02:28:24+00:00,BasedTradeZ,I wasn't planning on taking any Longs.. Howeve...
14997,2022-06-07 02:28:18+00:00,IGSquawk,Clearly they should raise bitcoin rates to sto...
14998,2022-06-07 02:28:16+00:00,peb310,@BitcoinComfy @begleristirner So is bitcoin…\n...


In [21]:
from gensim.parsing.preprocessing import remove_stopwords


In [22]:
import nltk
nltk.download('vader_lexicon')
nltk.download('words')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gunin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\gunin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [25]:
sia = SIA()
from itertools import chain
import regex as re


def clean_text(input_str: str) -> str:
    input_str = input_str.lower()
    input_str = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", input_str)
    input_str = " ".join(input_str.split())
    input_str = re.sub("[^A-Za-z0-9]"," ",input_str)
    input_str = re.sub(r'^https?:\/\/.*[\r\n]*', '', input_str, flags=re.MULTILINE)
    input_str = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', input_str)
    input_str = re.sub(r"www.\S+",'',input_str)
    input_str = remove_stopwords(input_str)
    input_str = ''.join([i for i in input_str if not i.isdigit()])
    return input_str

def build_sentiment_df(tweet_dataframe, target_column):
    # getting the corresponding data in lists
    raw_tweets = tweet_dataframe[target_column].tolist()
    cleaned_tweets = [clean_text(i) for i in raw_tweets]
    nltk_polarity = [sia.polarity_scores(i)['compound'] for i in cleaned_tweets]
    
    # building the dataframe
    final_df = tweet_dataframe.copy()
    final_df['cleaned_tweets'] = cleaned_tweets
    final_df['nltk_polarity'] = nltk_polarity

    final_df = final_df[final_df['Tweet'].apply(lambda x: len(x.split(',')) < 280)]
    final_df = final_df.drop_duplicates()
    final_df = final_df[final_df['Tweet'].str.contains("rt ") == False]
    final_df = final_df[final_df['nltk_polarity'] != 0.0000] 

    return final_df

In [26]:
btc_data = build_sentiment_df(df, "Tweet")

In [27]:
btc_data

,Date,User,Tweet,cleaned_tweets,nltk_polarity
1,2022-06-07 09:16:11+00:00,MinerSalesUK,The best entry level #ASICMiner you can buy w...,best entry level asicminer buy breaking bank w...,0.7964
3,2022-06-07 09:16:09+00:00,bmurphypointman,#affiliate #tumblr #twitter #facebook #instagr...,affiliate tumblr twitter facebook instagram we...,0.4404
4,2022-06-07 09:16:08+00:00,bmurphypointman,#affiliate #bitcoin #tumblr #twitter #facebook...,affiliate bitcoin tumblr twitter facebook inst...,0.4404
8,2022-06-07 09:16:08+00:00,Amdtrades,"Even if Bitcoin pumps to $35,000 that would ac...",bitcoin pumps actually set altcoins crushed ...,-0.4019
9,2022-06-07 09:16:07+00:00,LivePriceCrypto,#Bitcoin Last Price $29591 #BTC 🚀\nDaily Indic...,bitcoin price btc daily indicators rsi ma ...,-0.2960
...,...,...,...,...,...
14994,2022-06-07 02:28:26+00:00,Sanjay_rana_,"""CertiK shares security tips following third B...",certik shares security tips following bayc sec...,0.9260
14995,2022-06-07 02:28:26+00:00,exchange_whale,"🐳 1,145,224 #USDT worth of #Bitcoin were sold ...",usdt worth bitcoin sold large orders minutes,0.2263
14996,2022-06-07 02:28:24+00:00,BasedTradeZ,I wasn't planning on taking any Longs.. Howeve...,wasn t planning taking longs bitcoin support l...,0.7783
14997,2022-06-07 02:28:18+00:00,IGSquawk,Clearly they should raise bitcoin rates to sto...,clearly raise bitcoin rates stop selloff,0.1280


In [29]:
btc_data.reset_index()
btc_data = btc_data.set_index("Date")
btc_data

,User,Tweet,cleaned_tweets,nltk_polarity
Date,,,,
2022-06-07 09:16:11+00:00,MinerSalesUK,The best entry level #ASICMiner you can buy w...,best entry level asicminer buy breaking bank w...,0.7964
2022-06-07 09:16:09+00:00,bmurphypointman,#affiliate #tumblr #twitter #facebook #instagr...,affiliate tumblr twitter facebook instagram we...,0.4404
2022-06-07 09:16:08+00:00,bmurphypointman,#affiliate #bitcoin #tumblr #twitter #facebook...,affiliate bitcoin tumblr twitter facebook inst...,0.4404
2022-06-07 09:16:08+00:00,Amdtrades,"Even if Bitcoin pumps to $35,000 that would ac...",bitcoin pumps actually set altcoins crushed ...,-0.4019
2022-06-07 09:16:07+00:00,LivePriceCrypto,#Bitcoin Last Price $29591 #BTC 🚀\nDaily Indic...,bitcoin price btc daily indicators rsi ma ...,-0.2960
...,...,...,...,...
2022-06-07 02:28:26+00:00,Sanjay_rana_,"""CertiK shares security tips following third B...",certik shares security tips following bayc sec...,0.9260
2022-06-07 02:28:26+00:00,exchange_whale,"🐳 1,145,224 #USDT worth of #Bitcoin were sold ...",usdt worth bitcoin sold large orders minutes,0.2263
2022-06-07 02:28:24+00:00,BasedTradeZ,I wasn't planning on taking any Longs.. Howeve...,wasn t planning taking longs bitcoin support l...,0.7783


In [34]:
btc_minute_sentiment = pd.DataFrame(btc_data.resample('1Min').mean()['nltk_polarity'])

In [35]:
btc_minute_sentiment

,nltk_polarity
Date,
2022-06-07 02:28:00+00:00,0.299659
2022-06-07 02:29:00+00:00,0.072579
2022-06-07 02:30:00+00:00,-0.073070
2022-06-07 02:31:00+00:00,0.060260
2022-06-07 02:32:00+00:00,0.210532
...,...
2022-06-07 09:12:00+00:00,0.201164
2022-06-07 09:13:00+00:00,0.015468
2022-06-07 09:14:00+00:00,0.230189
